In [ ]:
import numpy as np
import scipy.signal as signal
import plotly.express as px
import pandas as pd # pour les dataframes
import sounddevice as sd # pour jouer les sons associés aux signaux

## 1 Fenêtrage temporel
1. Décrire le fonctionnement de la fonction np.fft.fftshift() , et son intérêt pour l’analyse spectrale. Dé-
terminer la transformation à appliquer sur l’axe fréquentiel pour afficher de manière cohérente le spectre obtenu
après l’application de la fonction np.fft.fftshift() .
2. Grâce aux fonctions signal.windows.hann() et signal.windows.boxcar() , créer deux sé-
quences wH et wB composées de K = 32 points correspondant respectivement aux fenêtres de Hann et rectan-
gulaire.
Afficher ces deux fenêtres en fonction du temps.

In [ ]:
np.fft.fftshift()
signal.windows.hann()
signal.windows.boxcar()